## Star Tracker ML Pipeline ##

In [15]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.1


In [2]:
# Configuration values
validation_split = 0.20
input_size = 10
num_epochs = 50
batch_size = 40
saved_model_dir = 'models/'

In [3]:
file = "../images_data/mag5_adverserial_bins/mag5_adverserial_bins.csv"
# TODO: read the header instead of hardcoding it
dataframe = pd.read_csv(file, header=0, names=["HIP", "bin0", "bin1", "bin2", "bin3", "bin4", "bin5", "bin6", "bin7", "bin8", "bin9"])
dataset = dataframe.values
X = dataset[:,1:11].astype(int)
Y = dataset[:,0].astype(int)
print("Input Size:", len(X[0]))
num_classes = len(np.unique(Y))
print("Output Size:", num_classes)

Input Size: 10
Output Size: 1608


In [9]:
# Encode class values as integers and then create dataframe with one hot encoding
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

dummy_Y = tf.one_hot(encoded_Y, depth=num_classes)
print(dummy_Y)



# Unused code from when I was sanity checking
"""
for i in range(100):
    if not (encoder.classes_[dummy_Y.numpy()[i].argmax()] == Y[i]):
        print("Error")
        # print(dummy_Y.numpy()[i].argmax())
        # print(Y[i])
        # print(encoder.classes_[dummy_Y.numpy()[i].argmax()])
"""

"""
# star_labels = np.zeros(num_classes).astype(int)
# get labels in num_classes sized list

# one_hot_array = dummy_Y.numpy()
# # print(Y[0])
# for i in range(len(one_hot_array)):
#     if star_labels[one_hot_array[i].argmax()] == 0:
#         star_labels[one_hot_array[i].argmax()] = Y[i]
# np.set_printoptions(threshold=np.inf)
# print(star_labels)
"""


tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(17688, 1608), dtype=float32)


'\n# star_labels = np.zeros(num_classes).astype(int)\n# get labels in num_classes sized list\n\n# one_hot_array = dummy_Y.numpy()\n# # print(Y[0])\n# for i in range(len(one_hot_array)):\n#     if star_labels[one_hot_array[i].argmax()] == 0:\n#         star_labels[one_hot_array[i].argmax()] = Y[i]\n# np.set_printoptions(threshold=np.inf)\n# print(star_labels)\n'

In [10]:
# Create the ML Model
star_model = tf.keras.Sequential([
  layers.Dense(input_size, activation="relu", name="layer1"),
  layers.Dense(64, activation="relu", name="layer2"),
  layers.Dense(32, activation="relu", name="layer3"),
  layers.Dense(1608, activation="softmax", name ="layer4")
])

star_model.compile(loss = "categorical_crossentropy",
                      optimizer = 'adam', metrics = ['accuracy'])

In [11]:
# Model Training
star_model.fit(X, dummy_Y, epochs=num_epochs)

Epoch 1/50


I0000 00:00:1719977164.184571  113436 service.cc:145] XLA service 0x7fc0c0004c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719977164.184762  113436 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-07-02 20:26:04.235318: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-02 20:26:04.413449: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1719977165.479121  125136 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 48 bytes spill stores, 48 bytes spill loads

I0000 00:00:1719977165.624499  125132 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 304 bytes spill stores, 304 bytes spill loads


112/553 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0016 - loss: 7.4054  

I0000 00:00:1719977166.797750  113436 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


552/553 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0202 - loss: 6.7107

I0000 00:00:1719977168.384025  125244 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1719977168.450889  125245 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_376', 300 bytes spill stores, 300 bytes spill loads



553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0203 - loss: 6.7065
Epoch 2/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3303 - loss: 2.5954
Epoch 3/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5934 - loss: 1.3788
Epoch 4/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7382 - loss: 0.8657
Epoch 5/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8086 - loss: 0.6364
Epoch 6/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8507 - loss: 0.4967
Epoch 7/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8676 - loss: 0.4480
Epoch 8/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8871 - loss: 0.3906
Epoch 9/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8958 - loss: 0.3500
Epoch 10/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9003 - loss: 0.3287
Epoch 11/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9054 - loss: 0.3293
Epoch 12/50
553/553 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

In [30]:
# Save Model

star_model.save(saved_model_dir + 'star_tracker_model_large.keras')
if not os.path.exists(saved_model_dir + 'star_tracker_saved_model/'):
    os.makedirs(saved_model_dir + 'star_tracker_saved_model/')
star_model.export(saved_model_dir + 'star_tracker_saved_model/')

# TODO: tflite conversion is not working.
# # Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir + 'star_tracker_saved_model/') # path to the SavedModel directory
tflite_model = converter.convert()

# # Save the model.
with open(saved_model_dir + 'star_tracker_model_small.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: models/star_tracker_saved_model/assets


INFO:tensorflow:Assets written to: models/star_tracker_saved_model/assets


Saved artifact at 'models/star_tracker_saved_model/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1608), dtype=tf.float32, name=None)
Captures:
  140468450171136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470127857152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470127856976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243647216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243655312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243651440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243660592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140467243661648: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1719977882.345787  113281 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1719977882.345828  113281 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-07-02 20:38:02.346039: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/star_tracker_saved_model/
2024-07-02 20:38:02.346465: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-02 20:38:02.346472: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/star_tracker_saved_model/
2024-07-02 20:38:02.351013: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-07-02 20:38:02.369465: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: models/star_tracker_saved_model/
2024-07-02 20:38:02.374925: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load for tags { serve }; Status: success: OK. Took 28888 microsecond